In [20]:
from transformers import Wav2Vec2Processor, Wav2Vec2Model
import librosa
import torch
import pandas as pd
import os
import random
import pickle
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [21]:
#dataset
folder = r"/content/drive/MyDrive/Audio_Speech_Actors_01-24/Actor_01"
path = r"/content/drive/MyDrive/Audio_Speech_Actors_01-24/Actor_01"
# where emotion word embeddings are stored
fasttext_folder='/content/drive/MyDrive/emotion_vectors'
#where to save the model
model_save = 'advanced_embedding_mapper.pth'
#where the pickle is saved
pickle_path = '/content/drive/MyDrive/USER/fileTensorDict.pckl'

In [22]:
def get_emotion_vector(filename):
    parts = filename.split('-')
    third_number = parts[2]

    emotion_vector_label = None
    if third_number == '05':
        emotion_vector_label = 'angry'
    elif third_number == '02':
        emotion_vector_label = 'calm'
    elif third_number == '07':
        emotion_vector_label = 'disgust'
    elif third_number == '06':
        emotion_vector_label = 'fearful'
    elif third_number == '03':
        emotion_vector_label = 'happy'
    elif third_number == '01':
        emotion_vector_label = 'neutral'
    elif third_number == '04':
        emotion_vector_label = 'sad'
    elif third_number == '08':
        emotion_vector_label = 'surprised'

    return emotion_vector_label

In [23]:
def addToDict(folder):
  emo_dict = {}
  for file in os.listdir(folder):
      emo_dict[file] = get_emotion_vector(file)
  str_emo_dict = str(emo_dict)
  with open("vectors.txt", "a") as vec:
    vec.write(str_emo_dict)

for x in os.listdir(folder):
  addToDict(folder)

In [24]:
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")
model = Wav2Vec2Model.from_pretrained("facebook/wav2vec2-base-960h")


Some weights of Wav2Vec2Model were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1', 'wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [25]:
def get_vector_from_audio(path):
  audio_input, sampling_rate = librosa.load(path, sr=16000)
  inputs = processor(audio_input, sampling_rate=16000, return_tensors="pt", padding=True)
  with torch.no_grad():
    outputs = model(**inputs)

  hidden_states = outputs.last_hidden_state

  vector_rep = torch.mean(hidden_states, dim=1)
  return vector_rep



In [26]:
def get_embeddings():
  filename_vector_dict = {}
  # path = r"/content/drive/MyDrive/Audio_Speech_Actors_01-24/Actor_01"
  for f in os.listdir(path):
    file_path = os.path.join(path,f)
    emov = get_vector_from_audio(file_path)
    filename_vector_dict[f] = emov
  return filename_vector_dict

In [27]:
def load_fasttext_embedding(emotion_label, fasttext_folder):
  filepath = os.path.join(fasttext_folder, f'{emotion_label}.txt')
  if not os.path.exists(filepath):
      raise FileNotFoundError(f"Embedding file for {emotion_label} not found in {folder}")

  embedding = []
  with open(filepath, 'r') as file:
      for line in file:
          embedding.append(float(line.strip()))

  return embedding


In [28]:
def map_fasttext_to_wav2vec(wav2vec_dict, fasttext_folder='emotion_vectors'):
    fasttext_vector_dict = {}
    for filename, wav2vec_embedding in wav2vec_dict.items():
        emotion_label = get_emotion_vector(filename)
        fasttext_embedding = load_fasttext_embedding(emotion_label, fasttext_folder)
        fasttext_embedding = torch.tensor(fasttext_embedding, dtype=torch.float32)
        fasttext_vector_dict[filename] = (wav2vec_embedding, fasttext_embedding)
    return fasttext_vector_dict

In [12]:
# filename_vector_dict = get_embeddings()
# vector_map = map_fasttext_to_wav2vec(filename_vector_dict,fasttext_folder)

In [29]:
def check_or_create_vector_map(pickle_path, fasttext_folder):
    if os.path.exists(pickle_path):
        with open(pickle_path, 'rb') as f:
            data = pickle.load(f)
            if 'vector_map' in data:
                print("vector_map loaded from pickle file.")
                return data['vector_map']

    # If the file does not exist or vector_map is not in the file, create it
    filename_vector_dict = get_embeddings()
    vector_map = map_fasttext_to_wav2vec(filename_vector_dict, fasttext_folder)

    # Save the vector_map to the pickle file
    with open(pickle_path, 'wb') as f:
        pickle.dump({'vector_map': vector_map}, f)
    print("vector_map created and saved to pickle file.")

    return vector_map

vector_map = check_or_create_vector_map(pickle_path, fasttext_folder)

vector_map created and saved to pickle file.


In [30]:
def split_sets(dictionary, train_ratio=0.8, seed=42):
    # Set the random seed for reproducibility
    random.seed(seed)

    # Get the list of keys and shuffle them
    dict_keys = [key for key in dictionary.keys()]
    random.shuffle(dict_keys)

    # Determine the split index
    split_index = int(len(dict_keys) * train_ratio)

    # Split the keys into training and test sets
    train_keys = dict_keys[:split_index]
    test_keys = dict_keys[split_index:]

    # Create training and test dictionaries
    train_dict = {key: dictionary[key] for key in train_keys}
    test_dict = {key: dictionary[key] for key in test_keys}

    return train_dict, test_dict


In [35]:
def getSets(vector_map):
  train_dict, test_dict = split_sets(vector_map)
getSets(vector_map)

In [43]:
class AdvancedEmbeddingMapper(nn.Module):
    def __init__(self):
        super(AdvancedEmbeddingMapper, self).__init__()
        self.fc1 = nn.Linear(768, 512)
        self.bn1 = nn.BatchNorm1d(1)
        self.fc2 = nn.Linear(512, 256)
        self.bn2 = nn.BatchNorm1d(1)
        self.fc3 = nn.Linear(256, 128)
        self.bn3 = nn.BatchNorm1d(1)
        self.fc4 = nn.Linear(128, 300)
        self.dropout = nn.Dropout(0.3)  # Increase dropout rate for better regularization
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.fc1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.dropout(x)

        x = self.fc2(x)
        x = self.bn2(x)
        x = self.relu(x)
        x = self.dropout(x)

        x = self.fc3(x)
        x = self.bn3(x)
        x = self.relu(x)
        x = self.dropout(x)

        x = self.fc4(x)
        return x

In [37]:
def create_dataloader(data_dict, batch_size=32, shuffle=True):
    wav2vec_tensors = []
    fasttext_tensors = []

    for key in data_dict:
        wav2vec_tensors.append(data_dict[key][0])
        fasttext_tensors.append(data_dict[key][1])

    X = torch.stack(wav2vec_tensors)
    Y = torch.stack(fasttext_tensors)

    dataset = TensorDataset(X, Y)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=shuffle)

    return dataloader

train_dataloader = create_dataloader(train_dict)
test_dataloader = create_dataloader(test_dict, shuffle=False)

In [44]:

model = AdvancedEmbeddingMapper()

# Define loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


def train_model(model, train_dataloader, num_epochs=50):
    for epoch in range(num_epochs):
        model.train()
        epoch_loss = 0
        for batch_x, batch_y in train_dataloader:
            optimizer.zero_grad()
            outputs = model(batch_x)
            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()

        if (epoch + 1) % 5 == 0:
            print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {epoch_loss / len(train_dataloader):.4f}')

train_model(model, train_dataloader)

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([32, 300])) that is different to the input size (torch.Size([32, 1, 300])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([16, 300])) that is different to the input size (torch.Size([16, 1, 300])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch [5/50], Loss: 0.0615
Epoch [10/50], Loss: 0.0385
Epoch [15/50], Loss: 0.0287
Epoch [20/50], Loss: 0.0242
Epoch [25/50], Loss: 0.0200
Epoch [30/50], Loss: 0.0182
Epoch [35/50], Loss: 0.0164
Epoch [40/50], Loss: 0.0147
Epoch [45/50], Loss: 0.0132
Epoch [50/50], Loss: 0.0127


Why do I get **"running_mean should contain 1 elements not 512"** when BatchNorm1d(512) or 256 or 128?

In [39]:
def evaluate_model_more(model, test_dataloader):
    model.eval()
    test_loss = 0
    all_targets = []
    all_predictions = []

    with torch.no_grad():
        for batch_x, batch_y in test_dataloader:
            outputs = model(batch_x).squeeze(1)  # Squeeze to remove singleton dimension
            loss = criterion(outputs, batch_y)
            test_loss += loss.item()

            all_targets.append(batch_y)
            all_predictions.append(outputs)

    # Compute average test loss
    test_loss /= len(test_dataloader)

    # Concatenate all targets and predictions
    all_targets = torch.cat(all_targets).cpu().numpy()
    all_predictions = torch.cat(all_predictions).cpu().numpy()

    # Compute additional metrics
    mse = mean_squared_error(all_targets, all_predictions)
    mae = mean_absolute_error(all_targets, all_predictions)
    r2 = r2_score(all_targets, all_predictions)

    print(f'Test Loss: {test_loss:.4f}')
    print(f'Mean Squared Error (MSE): {mse:.4f}')
    print(f'Mean Absolute Error (MAE): {mae:.4f}')
    print(f'R-squared (R²): {r2:.4f}')

evaluate_model_more(model, test_dataloader)

Test Loss: 0.0038
Mean Squared Error (MSE): 0.0038
Mean Absolute Error (MAE): 0.0470
R-squared (R²): -0.4503
